In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

**Решение задачи из пункта а.)**

In [11]:
h_0 = 0.01
l_x = 1
l_y = np.sqrt(2)

n_x = int(np.round(l_x / h_0))
n_y = int(np.round(l_y / h_0))

h_x = l_x / n_x
h_y = l_y / n_y

x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

X, Y = np.meshgrid(x, y, indexing='ij')

In [3]:
def f(x, y):
  return np.sin(5 * x) * ((25 + np.pi ** 2 * y ** 2) * np.cos(np.pi * y ** 2 / 2) + np.pi * np.sin(np.pi * y ** 2 / 2))


def μ_L(y):
  return 0


def μ_R(y):
  return np.sin(5) * np.cos(np.pi * y ** 2 / 2)


def μ_B(x):
  return np.sin(5 * x)


def μ_T(x):
  return -np.sin(5 * x)


def u_0(x, y):
  return np.sin(5 * x) * np.cos(np.pi * y ** 2 / 2)

In [4]:
def F(X, Y, h_x, h_y):
    res = f(X, Y)

    # i = 1
    res[0, :] += 2 * μ_L(Y[0, :]) / h_x ** 2

    # i = n_x
    res[-1, :] += 2 * μ_R(Y[-1, :]) / h_x ** 2

    # j = 1
    res[:, 0] += 2 * μ_B(X[:, 0]) / h_y ** 2

    # j = n_y
    res[:, -1] += 2 * μ_T(X[:, -1]) / h_y ** 2

    return res


def L(u, h_x, h_y):
    res = np.zeros_like(u)

    # i > 1
    res[1:, :] += (u[1:, :] - u[:-1, :]) / h_x ** 2

    # i = 1
    res[0, :] += 2 * u[0, :] / h_x ** 2

    # i < n_x
    res[:-1, :] += (u[:-1, :] - u[1:, :]) / h_x ** 2

    # i = n_x
    res[-1, :] += 2 * u[-1, :] / h_x ** 2

    # j > 1
    res[:, 1:] += (u[:, 1:] - u[:, :-1]) / h_y ** 2

    # j = 1
    res[:, 0] += 2 * u[:, 0] / h_y ** 2

    # j < n_y
    res[:, :-1] += (u[:, :-1] - u[:, 1:]) / h_y ** 2

    # j = n_y
    res[:, -1] += 2 * u[:, -1] / h_y ** 2

    return res

In [5]:
def minimal_residuals(h_x, h_y, e, max_iter):
    u = np.zeros_like(X)
    r = L(u, h_x, h_y) - F(X, Y, h_x, h_y)

    for i in range(max_iter):
        Lr = L(r, h_x, h_y)
        τ = (Lr * r).sum() / (Lr ** 2).sum()
        u += -r * τ
        r = L(u, h_x, h_y) - F(X, Y, h_x, h_y)
        
        if np.linalg.norm(r) < e:
            break
    return u,i

In [12]:
e = 1e-1
max_iter = 100000

u_exact = u_0(X, Y)
u_num, iter = minimal_residuals(h_x, h_y, e, max_iter)
print(iter)

20364


In [7]:
h_0 = 0.05
l_x = 1
l_y = np.sqrt(2)

n_x = int(np.round(l_x / h_0))
n_y = int(np.round(l_y / h_0))

h_x = l_x / n_x
h_y = l_y / n_y

x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

X, Y = np.meshgrid(x, y, indexing='ij')
e = 1e-1
max_iter = 100000

u_exact = u_0(X, Y)
u_num, iter = minimal_residuals(h_x, h_y, e, max_iter)
print(iter)

648


In [13]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Numerical solution", "Analytical solution"])

fig.add_trace(go.Heatmap(z=u_num.T, colorscale='Magma', x0=0, dx=l_x/n_x, y0=0, dy=l_y/n_y, colorbar=dict(title='u')), row=1, col=1)
fig.add_trace(go.Heatmap(z=u_exact.T, colorscale='Magma', x0=0, dx=l_x/n_x, y0=0, dy=l_y/n_y, colorbar=dict(title='u')), row=1, col=2)



fig.update_layout(
    width=1000,
    height=500,
)

fig.update_xaxes(title_text='x', row=1, col=1)
fig.update_xaxes(title_text='x', row=1, col=2)
fig.update_yaxes(title_text='y', row=1, col=1)
fig.update_yaxes(title_text='y', row=1, col=2)

fig.show()

In [14]:
fig = make_subplots(rows=1, cols=1)
fig.update_layout(
    width=1000,
    height=1100,
)
fig.add_trace(go.Heatmap(z=(np.abs(u_exact-u_num)).T, colorscale='Magma', x0=0, dx=l_x/n_x, y0=0, dy=l_y/n_y, colorbar=dict(title='u')), row=1, col=1)